In [6]:
import re
import time
from meituan_tools import Operation
import pandas as pd
import pyecharts

In [7]:
# 数据库连接
op = Operation()
conn = op.db_conn()
cur = conn.cursor()

+ **拿到数据先整体数值字段max,min看下异常数据，这些极值会影响最后统计结果<br />观察后发现人均价格和评分有部分为0，对最后统计没有意义，所以在这里我们需要这些数据进行剔除**

In [3]:
# 数据检查
sql = '''	select distinct 
	b.poi_id,b.title,b.avg_price,b.avg_score,b.comment_num,b.address from meituan_classify_info as a
	inner join meituan_shop_info as b on a.sub_id = b.sub_id and a.class_type = b.class_type
	where a.class_type = 1 and a.sub_id not in(24,393,395) and b.avg_price <> 0 and b.avg_score <> 0
	order by avg_price desc'''

df = pd.read_sql(sql,conn,index_col=['poi_id'])
df

,title,avg_price,avg_score,comment_num,address
poi_id,,,,,
162761320,黄公子,2665.0,5.0,7,徐汇区天平路320弄21号1-2层（衡山路花园洋房旁）
2718768,十里洋场江景会所（外滩店）,1489.0,4.0,0,虹口区黄浦路108号（近外白渡桥）
1578636,MOOK酒吧,1086.0,5.0,51,静安区南京西路758号汇银大厦5楼（近凤阳路）
1580796,浦江二号,994.0,5.0,2,浦东新区滨江大道2250号（近凯宾斯基酒店）
2473301,洋房火锅（新天地店）,903.0,5.0,510,黄浦区新天地南里黄陂南路376号（近兴业路）
476286,菲芘3d酒吧,883.0,4.0,22,徐汇区衡山路10号欧登大厦1楼
158735513,本色TureColor酒吧（外滩店）,846.0,5.0,60,黄浦区淮海路东段与柳林路交口柳林大厦六层
454005,宏豪浦江一号,819.0,5.0,5,浦东新区滨江大道3788号（近商城路）
40673951,原烹（金虹桥国际中心店）,790.0,5.0,3,长宁区茅台路179号金虹桥国际中心4层


In [4]:
from pyecharts import Liquid

liquid = Liquid(title="样本数",subtitle='样本总量29876 剔除均价和评分为0的数据所剩的数量26793')
liquid.add("Liquid", [0.89, 0.7, 0.5, 0.3], is_liquid_outline_show=False)
liquid

In [5]:
df = df.head(10)
title = df['title']
avg_price =  df['avg_price']
avg_score = df['avg_score']
comment_num = df['comment_num']

from pyecharts import Bar,Grid

bar = Bar(title="餐饮商家前十排行",subtitle='数据来源神秘组织：*团',width=800,height=400)
bar.add("人均", title, avg_price, mark_point=["min", "max"],mark_line=["average"])
bar.add("评论数", title, comment_num,mark_point=["max"],is_label_show=True, xaxis_rotate=30)

grid = Grid(height=500)
grid.add(bar, grid_bottom="30%")

grid

+ **对人均降序排序后，发现最贵的是“黄公子”人均2665，不由得让我们对前10的商家起了兴趣，于是上了百度查看了一下**

>+ **[【黄公子】](http://s.dianping.com/topic/33883578?utm_source=forum_pc_index)<br />
隐藏在老式洋房里的一家人均千元的定制私房菜，仪式感十足，每天只接受10位顾客的预定，简友们谁家宽裕的可以带我去遛遛 =。=**

<div style="float:left;clear:both ;solid 000;margin:2px;">
    <img src="https://p1.meituan.net/dpgroup/089fb645a439d5112f54709464198ebc2258883.jpg" height="300" width="180"/>
</div>    
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="https://p0.meituan.net/dpgroup/08c6410707c7ee013fe42f49367a0f873015366.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="https://p1.meituan.net/dpgroup/0d0c119c8c5ed617733543dbc6c814571009636.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="https://p1.meituan.net/dpgroup/b7be7517245ca663efe94ac6d9c52ed22526430.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="https://p1.meituan.net/dpgroup/6285d273f31ab98e1cbe5061747095ad2247619.jpg" height="300" width="180"/>
</div> 

>+ **[【MOOK酒吧】](http://www.dianping.com/shop/5583053)<br />什么是“当当当当，当当当?” 天哪! 还有漂亮的小姐姐...~哇靠~仿佛恋爱了... =。=**

<div style="float:left;clear:both ;solid 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/fh2O7aK9LdOvLZsPRasECbn4uJsOpNAw0yd7AzdTGAf9FiMxfamhzOpBpUB36ZKETYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div>    
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/CvQummZpOhNfK0n2sPv8sFcmkPzo3yecpXgwr1Lrzd4kNVqbowOSz6WrD4VBGHSxTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/Kjehf2z1DpAYJShzXNEcud6V7W2TsUTmYRErB8FI7s44OnA797ve0qcViZcSKwMVTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/PMC7S3ty3JH3sw-xVP93x3TSsjarY5GxYx-gycoJzhsru7nMSCtlSj7iGU77jW6uTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/DdbjvXy730rUs9XFo8gqiZ29HqXj0m1tsQE87NFZxXMTsu-eL1pFIl8y1JR6Gd7YTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 

>+ **[【洋房火锅】](http://www.dianping.com/shop/8844957)<br />火锅中的劳斯莱斯...A级和牛600~800,涮一片牛肉小两百...我的天...贫穷真的限制我了我的想象 =。=**

<div style="float:left;clear:both ;solid 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/1qJi9p-wLeonVl6Y9WiyFPSanre0MLkueCtCk25iI7w85m6KLxJrZMuRwDiePBFkTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div>    
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/o1Jnmy2bPPChRikogYoVC-pw_btvmkBBrWfbZefBAN0kNVqbowOSz6WrD4VBGHSxTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/exfn_RFUtZ0im46Rl-MXHedTYeJbsQERCP5g57Wid75WzN_4hUPhHPENFzwE2h71TYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/6KngG1CJCk6uC6Ym2VWxmCzc5JFFiQKAAXEcmK-fA_t2Z2NWo8mGzuxwarQuIULZTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/n_irq9QArm3UMjp0K2j5595shc1P2PGhOOx3gWR1N6gpJ36aly4bqURKSnWlKZwiTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 

+ **看完最贵的，那么我们看下最热的，也就是评论数最多的**

In [6]:
sql3 = '''#sql3
select distinct a.sub_id,a.sub_name,
b.poi_id,b.title,b.avg_price,b.avg_score,b.comment_num,b.address from meituan_classify_info as a
inner join meituan_shop_info as b on a.sub_id = b.sub_id and a.class_type = b.class_type
where a.class_type = 1 and a.sub_id not in(24,393,395) and b.avg_price <> 0 and b.avg_score <> 0 
and CONCAT(b.sub_id,b.poi_id) not in ('6342030772','4050576755','6350576755','4052163162','2006068147006','5452800270','4087812358','6387812358','543311762')
order by b.comment_num desc limit 10;
'''
df3 = pd.read_sql(sql3,conn)
df3

,sub_id,sub_name,poi_id,title,avg_price,avg_score,comment_num,address
0,36,小吃快餐,4091974,方燕烤猪蹄店（罗宾森店）,10.0,4.5,24465,嘉定区城中路138号罗宾森广场南侧2号美食亭
1,40,自助餐,5396841,多伦多海鲜自助餐厅（江桥万达店）,99.0,3.9,20903,嘉定区华江路848号江桥万达广场百货楼5楼5001铺位（金运路地铁站东）
2,40,自助餐,42030772,上海贻贝湾海鲜自助餐厅(百联南桥购物中心店),95.0,3.7,19766,奉贤区百齐路588号百联南桥购物中心二期5层
3,36,小吃快餐,5565604,西安章记（曹路宝龙广场店）,18.0,4.8,17251,浦东新区金海路2505号曹路宝龙广场一期1层
4,36,小吃快餐,4137469,盘古烤猪蹄（鹿都总店）,13.0,4.8,15752,松江区庙前街98号（面潮未来旁边）
5,17,火锅,68147006,牛市缘潮汕鲜牛肉火锅（乐颂坊店）,92.0,3.5,14511,松江区松汇中路568号乐颂坊3楼
6,40,自助餐,52800270,炭香园自助烧烤（人民北路店）,57.0,3.5,13116,松江区人民北路112号三楼（招商银行楼上）
7,36,小吃快餐,1584738,粉面桃花土豆粉（天钥桥路店）,24.0,4.2,12774,徐汇区天钥桥路93-3号（近美罗城）
8,40,自助餐,3311762,圣火营BBQ（南京东路店）,57.0,3.5,12276,黄浦区南京东路588号丝绸商厦6楼（世纪广场对面）
9,40,自助餐,40833128,鑫海汇海鲜烤肉火锅自助餐厅（巴黎春天天山店）,70.0,3.3,12110,长宁区天山路762号巴黎春天中区4层


+ **从数据表中可以看出，整个评论排行榜的前十均被 小吃快餐、自助餐 2类霸榜（其中还剔除了各种xxx分店），有趣的是前十店铺的地址大多都在 嘉定、奉贤、松江、曹路 这些地区都在上海外环以外，离市区是相当远，兄die!我想去吃吃验证下都心有余而力不足。最后这两类分类到底是否真的是客量大，还是存在刷榜，或是神秘组织\*团商铺分类关系，个人就不深度去研究了...太花费时间，但我真的很好奇~知道的简友们请留言...**

In [7]:
from pyecharts import Bar,Grid

data = sorted(df3[['title','comment_num']].values ,key=lambda x: x[1],reverse=True)
attr = [i[0] for i in data]
val = [i[1] for i in data]

bar = Bar()
bar.add('comment_num', attr, val,is_label_show=True,xaxis_rotate=30)
grid3 = Grid(height=500)
grid3.add(bar, grid_bottom="30%")

grid3

> [【燕烤猪蹄店】](http://www.dianping.com/shop/58919293)
>+ **评论数量第一的竟然是家烤猪蹄店，个人是超级喜欢吃猪蹄的，皮脆肉软还美颜...Duang!!但为何以如此高的评论数位居榜首？对这块了解的请留言告知~(喜欢图3的请为我点赞!)**

<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/ktUEW5kKJiEidB7kDqF_j1a0z62NhOm5YlkD4uLuOaxF3HNB3vggzmw8raY1JNntTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="330"/>
</div>
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/Evr544EhAIuulJNnEolo1ADBfIGYusZQGC9aZIW_qG4LPJtbh_j3JRHoujrHxBzGTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="330"/>
</div>    
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/Qc0poBNvPAog_0wiqSoRscCswJHzgdj1ZJ52VUVRRPJJZLFfMo2DngJBqjtlKhWfTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="300"/>
</div> 



+ **对餐厅感兴趣的可以点餐厅链接↑↑↑ <br /> <br />好了，言归正传，分类不同，价格会相差很大，所以不能对所有类别进行全量统计（例如日料和小吃的价格就是天壤之别）下面的箱线图就很好的表达了这些分类的数据分布情况（ps：箱线图可以表达很多基础分布,最上线表示最大，最下线表示最小，箱体内表示整体中正常数据，箱体外表示整体中部分突变数据，箱体中线表示中分为，箱体上框表示第一四分位，箱体下框表示第三四分位）**

In [8]:
from pyecharts import Boxplot

sql = '''select distinct sub_id,sub_name from meituan_classify_info where class_type = 1 and sub_id not in(24,393,395)'''
df = pd.read_sql(sql,conn)

sql2 = '''select distinct poi_id,avg_price,sub_id from meituan_shop_info where class_type = 1 and avg_price <> 0 and avg_score <> 0'''
df2 = pd.read_sql(sql2,conn)

In [9]:
x_axis = []
y_axis = []
for i in df.index:
    sub_id = df.loc[i].values[0]
    sub_name = df.loc[i].values[1]  
    avg_price = df2[df2['sub_id'] == sub_id ]['avg_price'].values
    x_axis.append(sub_name)
    y_axis.append(avg_price)
    
boxplot = Boxplot("菜系数据分布情况")
_yaxis = boxplot.prepare_data(y_axis)   # 转换数据
tp_dict = {k:v for k,v in zip(x_axis,_yaxis)}
idx_tp = sorted([(max(v),k) for k,v in zip(x_axis,_yaxis)])

# 根据max排序
x_ax = []
y_ax = []
for i in idx_tp:
    x_ax.append(i[1])
    y_ax.append(tp_dict[i[1]])
boxplot.add('boxplot', x_ax,y_ax,is_datazoom_show=True, datazoom_type='both',xaxis_rotate=30)

grid = Grid()
grid.add(boxplot, grid_bottom="20%")
grid

+ **分类有很多种，要把菜系和食品的类别区分开（火锅可以是川菜也可以是日料，日料可以是自助也可以是海鲜）所以我们要把分类再细化，这里剔除食品分类，筛选出菜系类别（如：日料、川菜、粤菜、浙江菜、西北菜等）**

In [10]:
sql4 = '''select sub_id,sub_name ,count(*) as cnt from (
	select distinct a.sub_id,a.sub_name,
	b.poi_id,b.title,b.avg_price,b.avg_score,b.comment_num,b.address,b.stamp from meituan_classify_info as a
	inner join meituan_shop_info as b on a.sub_id = b.sub_id and a.class_type = b.class_type
	where a.class_type = 1 and a.sub_id in (55,56,57,58,59,60,62,227,233,20003,20059,20060)
    and b.avg_price <> 0 and b.avg_score <> 0 
) as x
group by sub_id,sub_name;
'''
df4 = pd.read_sql(sql4,conn,index_col='sub_id')
df4

,sub_name,cnt
sub_id,,
55,川湘菜,796
56,江浙菜,653
57,粤菜,578
58,西北菜,84
59,京菜鲁菜,68
60,云贵菜,81
62,东南亚菜,202
227,台湾/客家菜,97
233,新疆菜,79


>+ **下面就用饼图的变种南丁格尔玫瑰图皮一下，因为比较酷炫~ 但在实际应用中饼图并不能很好的诠释分类情况因为看面积有些近似的面积很难区分谁多谁少，还是柱状图来的直观**

In [11]:
from pyecharts import Pie

sorted_df = sorted(df4[['sub_name','cnt']].values, key=lambda x:x[1],reverse=True)
attr = [i[0] for i in sorted_df]
val = [i[1] for i in sorted_df]
    

pie = Pie("*团各大菜系店铺数", title_pos='center', width=800)

pie.add("菜系", attr, val, center=[50, 50], is_random=True,
        radius=[35,65], rosetype='radius',legend_orient='vertical',legend_pos='left',
        is_legend_show=True, is_label_show=True)
pie

+ **川湘菜、浙江菜、日料、粤菜、韩料 位居前5，可以说统计结果跟现实中完全吻合<br />继续对数据进行下钻，不才学过日语也了解过日本历史,所以对日料有着特别的青睐，接下来就来专门研究下日料的情况。**

>+ **插句题外话,若不是战争的原因日本是有着高度文明的一个国家，经常告诉自己四个字“工匠精神”，日本人捏饭团捏一辈子，凿冰凿一辈子，炸天妇罗炸一辈子，工匠们喜欢不断雕琢自己的产品，不断改善自己的工艺，享受着产品在双手中升华的过程。工匠们对细节有很高要求，追求完美和极致，对精品有着执着的坚持和追求，把品质从0提高到1，其利虽微，却长久造福于世，它是极高的职业道德、职业能力、职业品质的体现。<br /><br />ps：科普科普个小知识，IPAD研磨工艺就是出自5个人小作坊——[小林研业](http://www.360doc.com/content/18/0205/23/51477542_728005923.shtml)<br /><div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1523125122660&di=5a9e8b194a63dd60a6776c45c84ead23&imgtype=0&src=http%3A%2F%2Fimg.mp.itc.cn%2Fupload%2F20160825%2F04d18149b910402cb79928bb2038f0fe_th.jpg"/>
</div> 
<br />这家为苹果研磨iPod的企业，就是位于日本新潟县燕市的“小林研业”，正是在这间其貌不扬的厂房里5名研磨技师花费了大约4年时间，为广受全球顾客欢迎的超过100万个“iPod”进行了背板的镜面加工。**

In [12]:
sql5 = '''select distinct a.sub_id,a.sub_name,
b.poi_id,b.title,b.avg_price,b.avg_score,b.comment_num,b.address,b.stamp from meituan_classify_info as a
inner join meituan_shop_info as b on a.sub_id = b.sub_id and a.class_type = b.class_type
where a.class_type = 1 and a.sub_id = 20059 and b.avg_price <> 0 and b.avg_score <> 0;
'''
df5 = pd.read_sql(sql5,conn,index_col='sub_id')
df5

,sub_name,poi_id,title,avg_price,avg_score,comment_num,address,stamp
sub_id,,,,,,,,
20059,日本料理,152515863,源季日本料理,115.0,5.0,93,虹口区东宝兴路118号,2018-04-06 21:57:36
20059,日本料理,466334,京都屋,78.0,3.0,0,黄浦区广西北路256号（近汉口路）,2018-04-06 21:54:52
20059,日本料理,158829945,牛繁烤肉,112.0,4.5,4,宝山区大华路大华一路大华虎城嘉年华B1楼352号B1楼108室,2018-04-05 17:35:54
20059,日本料理,397913,伊万里川,282.0,5.0,977,松江区人民北路1799号开元大酒店3楼（北翠路人民北路）,2018-04-08 02:20:34
20059,日本料理,1556061,和惠自助烧肉精致料理（川沙店）,164.0,4.0,2231,浦东新区川沙路5558号绿地东海岸国际广场2号楼1层,2018-04-08 02:19:43
20059,日本料理,5450114,友楽日本料理,202.0,3.8,693,松江区普照路1弄5号（普照大厦对面，靠近松汇中路）,2018-04-08 02:19:35
20059,日本料理,6336794,樱花いざかや居酒屋,86.0,4.0,288,浦东新区浦东南路2141弄4号(近浦澳路（浦东南路浦澳路路口）),2018-04-05 17:34:41
20059,日本料理,96210122,匠品精致料理,282.0,4.6,69,闵行区陈行公路2699号华侨城二期中央水岸二楼,2018-04-08 02:20:57
20059,日本料理,475185,御寿司（国定路店）,25.0,4.5,37,杨浦区国定路600弄40-1号（406公交站总站旁）,2018-04-05 17:34:41


<div style="float:left;clear:both ;solid 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/Rnmqu5D-Lc_Aj3XF6btESggR1j0zhuMR5MerxBOGvhZ6X-SGifYn7dkErBNhKP0ATYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div>    
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/grVIys6PnGRFd0KJXPM6D5L1cm2--ZIdVVrMzIafnrDVeCVtTYZQnZunqDYamz1ITYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/iT5hKDR-SZXcZZe6MdqzWZ51dDb3dh1RkHFdtqxUNZ4Ho8itsBkcmVncTI8kIYxjTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/f5sfzed5xqWpi4nm0DrEGhG4ZRZ-GRLhizP3zjGSngZ3FBJMSLaCv8bY782i7uhfTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 
<div style="float:left;border:solid 1px 000;margin:2px;">
    <img src="http://qcloud.dpfile.com/pc/UnXnR771C3W3EpvMi66Ow7Bsg8wmgjaDHqC2QmaIPHujpO0oDC1nRw8deAzYxv-zTYGVDmosZWTLal1WbWRW3A.jpg" height="300" width="180"/>
</div> 

黄浦区江西中路328号（南京东路新世界大丸百货）
showLocation&&showLocation({"status":0,"result":{"location":{"lng":121.49295308929035,"lat":31.244728849293325},"precise":1,"confidence":80,"level":"道路"}})
